In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [2]:
out='../output/'
inb='../input/flows/'
pref='idm'
inp = inb + pref + '/'

In [3]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-01-01,160,ZAMZ,0,ZAMC
1,2016-01-01,170,ZIM,0,ZAMZ
2,2016-01-01,160,ZAMZ,1,ZAMC
3,2016-01-01,170,ZIM,1,ZAMZ
4,2016-01-01,160,ZAMZ,2,ZAMC
...,...,...,...,...,...
13195,2023-04-30,-74,ZIM,23,ZIMA
13196,2023-04-30,-74,ZIMA,23,BOT
13197,2023-04-30,74,ZIMA,23,ZIM
13198,2023-04-30,0,ZIMA,23,MOZN_EDM


In [4]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

C:\Users\kabom\AppData\Local\Temp\ipykernel_35360\1849715270.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dam=dam.drop(['date'],1)


price  volume  year  month  day  day_of_week      value  cat
date       hour                                                               
2016-01-01 0      40.10   477.5  2016      1    1            4  19147.750  off
           1      40.08   486.6  2016      1    1            4  19502.928  off
           2      40.06   495.6  2016      1    1            4  19853.736  off
           3      40.08   496.6  2016      1    1            4  19903.728  off
           4      40.07   502.9  2016      1    1            4  20151.203  off
...                 ...     ...   ...    ...  ...          ...        ...  ...
2023-04-30 19    289.93    20.0  2023      4   30            6   5798.600  off
           20    289.93    38.0  2023      4   30            6  11017.340  off
           21     31.99    74.0  2023      4   30            6   2367.260  off
           22     32.00    74.0  2023      4   30            6   2368.000  off
           23     31.99    74.0  2023      4   30            6   2367.260  off

[63396 rows x 8 columns]

In [5]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

C:\Users\kabom\AppData\Local\Temp\ipykernel_35360\2288514685.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  flow=flow.dropna(0)


,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-01,0,160,ZAMZ,ZAMC,2016,1,1,4,40.10,6416.00
1,2016-01-01,0,170,ZIM,ZAMZ,2016,1,1,4,40.10,6817.00
2,2016-01-01,1,160,ZAMZ,ZAMC,2016,1,1,4,40.08,6412.80
3,2016-01-01,1,170,ZIM,ZAMZ,2016,1,1,4,40.08,6813.60
4,2016-01-01,2,160,ZAMZ,ZAMC,2016,1,1,4,40.06,6409.60
...,...,...,...,...,...,...,...,...,...,...,...
505633,2023-04-30,23,-74,ZIM,ZIMA,2023,4,30,6,31.99,-2367.26
505634,2023-04-30,23,-74,ZIMA,BOT,2023,4,30,6,31.99,-2367.26
505635,2023-04-30,23,74,ZIMA,ZIM,2023,4,30,6,31.99,2367.26
505636,2023-04-30,23,0,ZIMA,MOZN_EDM,2023,4,30,6,31.99,0.00


In [6]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2016-01-01,0,160,6416.00,40.10,ZAMC,2016,1,1,4
1,2016-01-01,0,170,6817.00,40.10,ZAMZ,2016,1,1,4
2,2016-01-01,1,160,6412.80,40.08,ZAMC,2016,1,1,4
3,2016-01-01,1,170,6813.60,40.08,ZAMZ,2016,1,1,4
4,2016-01-01,2,160,6409.60,40.06,ZAMC,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...
505633,2023-04-30,23,74,2367.26,31.99,ZIM,2023,4,30,6
505634,2023-04-30,23,74,2367.26,31.99,ZIMA,2023,4,30,6
505635,2023-04-30,23,-74,-2367.26,31.99,ZIMA,2023,4,30,6
505636,2023-04-30,23,0,-0.00,31.99,ZIMA,2023,4,30,6


In [7]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [8]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week    index  flow    value  \
date       hour                                                                
2016-01-01 0     ZAMC  2016      1    1            4        0   160  6416.00   
           0     ZAMZ  2016      1    1            4        1    10   401.00   
           0      ZIM  2016      1    1            4        1  -170 -6817.00   
           1     ZAMC  2016      1    1            4        2   160  6412.80   
           1     ZAMZ  2016      1    1            4        5    10   400.80   
...               ...   ...    ...  ...          ...      ...   ...      ...   
2023-04-30 22    ZAMC  2023      4   30            6  3033403   116  3712.00   
           22    ZAML  2023      4   30            6  1011152    32  1024.00   
           23     MOZ  2023      4   30            6  5056050  -148 -4734.52   
           23    ZAMC  2023      4   30            6  3033703   116  3710.84   
           23    ZAML  2023      4   30            6  1011252    32  1023.68   

                  price  cat  
date       hour               
2016-01-01 0      40.10  off  
           0      80.20  off  
           0      40.10  off  
           1      40.08  off  
           1      80.16  off  
...                 ...  ...  
2023-04-30 22    192.00  off  
           22     64.00  off  
           23    319.90  off  
           23    191.94  off  
           23     63.98  off  

[265356 rows x 10 columns]

In [9]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat      side                                       
2016 1     BOT  evening  supply        -25   -1501.40           2   
                morning  supply       -285  -14572.40           4   
                off      supply      -1270  -59474.14          45   
                standard supply      -1165  -57809.40          18   
           HCB  evening  supply        -57   -6204.59           2   
...                                    ...        ...         ...   
2023 4     ZAML off      supply       -246  -24711.58          13   
                standard consume       244    9128.72           8   
                         supply       -600  -91490.60          30   
           ZIM  off      consume     11364  403500.96          70   
                standard consume       340   12267.68           4   

                                  price_weight_mean  price_mean       power  
year month node cat      side                                                
2016 1     BOT  evening  supply           60.056000   60.050000  -12.500000  
                morning  supply           51.131228   55.290000  -71.250000  
                off      supply           46.830031   48.463556  -28.222222  
                standard supply           49.621803   56.768333  -64.722222  
           HCB  evening  supply          108.852456   99.140000  -28.500000  
...                                             ...         ...         ...  
2023 4     ZAML off      supply          100.453577  198.707692  -18.923077  
                standard consume          37.412787   75.070000   30.500000  
                         supply          152.484333  304.968667  -20.000000  
           ZIM  off      consume          35.506948  427.412571  162.342857  
                standard consume          36.081412  432.990000   85.000000  

[3347 rows x 6 columns]

In [10]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')